<a href="https://colab.research.google.com/github/lilmonteiro/fase3-techchallenge/blob/main/finetuning_describer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conexão com Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalação das dependencias

In [8]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install transformers datasets
!pip install xformers
!pip install trl
!pip install triton
!pip install peft
!pip install accelerate
!pip install bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ijtai3nz/unsloth_ec8251868a99491494dbe131797d62d9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ijtai3nz/unsloth_ec8251868a99491494dbe131797d62d9
  Resolved https://github.com/unslothai/unsloth.git to commit 6c534341bb229b136f9504443f0161645d2070c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 10.9 MB/s eta 0:00:00


## Importação das dependencias

Esse código define a configuração para carregar e treinar modelos de linguagem grandes em 4 bits, otimizando o uso de memória. Ele utiliza modelos da biblioteca unsloth e verifica o suporte a bfloat16 para maior eficiência de treino. A lista fourbit_models contém os modelos disponíveis para carregar, com comprimento máximo de sequência definido em 2048 tokens, ajustado para processamento eficiente.

In [10]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


OUTPUT_PATH_DATASET = onde será salvo o dataset depois de formatado

INPUT_PATH_DATASET = onde o dataset de treino está no drive

In [12]:
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/FIAP/TC4/formatted_dataset_asdict.json"
INPUT_PATH_DATASET = "/content/drive/MyDrive/FIAP/TC4/trn.json"

Caso não use o dataset do Drive, pode importar do hugginface ABAIXO:

In [14]:
dataset = load_dataset("lilmonteiro/amazon-titles", data_files="trn.json")

trn.json:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

## Formatação

Este código processa um dataset para transformá-lo em entradas de modelo. A função separate_text extrai o título e o conteúdo de um item do dataset e cria uma instrução padrão. O código percorre os dados de treino, separa o texto, e organiza as instruções, entradas e saídas em listas. Em seguida, armazena essas listas em um dicionário e salva o resultado em um arquivo JSON no caminho OUTPUT_PATH_DATASET, indicando que o dataset foi salvo com sucesso.

In [16]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        instruction = "GENERATE DESCRIPTIONS FOR THIS PRODUCTS"
        input_text = full_text['title']
        response = full_text['content']

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data['train']:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

## Filtrando do Dataset todos os contents vazios

In [18]:
filtered_dataset = dataset.filter(lambda example: example['content'] != '')

Filter:   0%|          | 0/2248619 [00:00<?, ? examples/s]

In [19]:
format_dataset_into_model_input(filtered_dataset)

Dataset salvo em /content/drive/MyDrive/FIAP/TC4/formatted_dataset_asdict.json


In [20]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Este código aplica um ajuste fino no modelo de linguagem usando o método *Parameter-Efficient Fine-Tuning* (PEFT) da biblioteca `FastLanguageModel`. Ele utiliza a técnica LoRA (Low-Rank Adaptation) para modificar apenas alguns parâmetros do modelo de forma eficiente, focando em módulos específicos como `q_proj`, `k_proj`, e outros.

Aqui estão os detalhes:
- `r = 16`: Define a classificação de adaptação de LoRA, controlando quantos parâmetros são treináveis.
- `target_modules`: Especifica os módulos do modelo nos quais LoRA será aplicado.
- `lora_alpha = 16`: Um hiperparâmetro que regula a escala dos parâmetros treinados.
- `lora_dropout = 0`: Define o *dropout* usado no ajuste de LoRA, e neste caso, ele está desativado.
- `bias = "none"`: Nenhum viés é ajustado durante o treinamento.
- `use_gradient_checkpointing = "unsloth"`: Ativa o *gradient checkpointing* para economizar memória durante o treino.
- `random_state = 3407`: Define uma semente para garantir a reprodutibilidade dos resultados.
- `use_rslora = False`: Indica que o *rslora* não será utilizado.
- `loftq_config = None`: Não usa uma configuração específica de *LoftQ*.

O objetivo geral é economizar memória e acelerar o treinamento, ajustando apenas partes específicas do modelo.

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Este código processa um dataset para gerar prompts no formato de um modelo de linguagem utilizando o template do Alpaca.

- **Template de Prompt**: A variável `alpaca_prompt` define o formato dos prompts, que contém uma instrução, um input adicional e a resposta.
  - O prompt segue o seguinte formato:
    ```
    ### Instruction:
    [instrução]

    ### Input:
    [input]

    ### Response:
    [resposta]
    ```
- **Função de formatação**: `formatting_prompts_func(examples)` formata cada exemplo no dataset. Ele pega as instruções, entradas e saídas, gera o prompt completo e adiciona o token de fim de sequência (`EOS_TOKEN`) ao final de cada prompt.
  - Para cada conjunto de `instruction`, `input`, e `output`, a função formata um texto e o adiciona à lista `texts`.
  - O resultado final é um dicionário com o campo `"text"` que contém os prompts formatados.
  
- **Carregamento e Mapeamento**:
  - O dataset é carregado usando `load_dataset` a partir de um arquivo JSON (definido por `OUTPUT_PATH_DATASET`).
  - O método `map` aplica a função de formatação a cada exemplo no dataset, processando-os em lotes.

Esse código é útil para preparar os dados de entrada e saída no formato necessário para treinar ou ajustar um modelo de linguagem.

In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

NameError: name 'tokenizer' is not defined

Esse código configura e inicializa um treinador (`SFTTrainer`) para ajustar um modelo de linguagem com um dataset específico, utilizando a técnica de ajuste fino supervisionado (Supervised Fine-Tuning).

Aqui estão os principais componentes:

- **Modelo e Tokenizer**:
  - `model`: O modelo de linguagem a ser treinado.
  - `tokenizer`: O tokenizer que processa o texto de entrada.

- **Dataset**:
  - `train_dataset`: O dataset utilizado para o treinamento, que já foi formatado anteriormente.
  - `dataset_text_field`: O campo no dataset que contém o texto processado para o modelo.
  
- **Configurações de Treinamento**:
  - `max_seq_length`: Define o comprimento máximo de sequência que o modelo pode processar.
  - `dataset_num_proc = 2`: Define o número de processos paralelos para processar o dataset.
  - `packing = False`: Desativa o "packing", que é a combinação de várias sequências em uma única.

- **TrainingArguments**:
  - `per_device_train_batch_size = 2`: Número de exemplos por batch no treinamento em cada dispositivo.
  - `gradient_accumulation_steps = 4`: Acumula gradientes por 4 passos antes de fazer uma atualização de peso.
  - `warmup_steps = 5`: Número de passos para o aquecimento do otimizador antes de ajustar os pesos.
  - `max_steps = 60`: Número total de passos de treinamento.
  - `learning_rate = 2e-4`: Taxa de aprendizado utilizada.
  - `fp16`: Treinamento em meia precisão (se `bfloat16` não for suportado).
  - `bf16`: Usa `bfloat16` se suportado.
  - `logging_steps = 1`: Log de informações a cada passo de treinamento.
  - `optim = "adamw_8bit"`: Otimizador AdamW com 8 bits, que economiza memória.
  - `weight_decay = 0.01`: Decaimento de peso para regularização.
  - `lr_scheduler_type = "linear"`: Scheduler linear para ajustar a taxa de aprendizado ao longo do tempo.
  - `seed = 3407`: Define uma semente para tornar o treinamento reprodutível.
  - `output_dir = "outputs"`: Diretório onde os resultados serão salvos.

Esse código treina o modelo de maneira eficiente, controlando parâmetros como memória e precisão numérica, otimizando o uso de recursos durante o treinamento.

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1498718 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.956700
2,3.085900
3,3.141800
4,2.936500
5,2.489200
6,2.692600
7,2.282000
8,2.142900
9,2.124400
10,1.806400


In [25]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "GENERATE DESCRIPTIONS FOR THIS PRODUCTS",
        "Catholics and Their Houses", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGENERATE DESCRIPTIONS FOR THIS PRODUCTS\n\n### Input:\nCatholics and Their Houses\n\n### Response:\nThis book is an excellent introduction to the history of the domestic church. It is a good read, easy to understand and well-written. It is a good introduction to the topic for those who are unfamiliar with it and is a good review for those who are already familiar with it. The book is also a good reminder to']

In [26]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "GENERATE DESCRIPTIONS FOR THIS PRODUCTS",
        "Catholics and Their Houses", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
GENERATE DESCRIPTIONS FOR THIS PRODUCTS

### Input:
Catholics and Their Houses

### Response:
This is a new and revised edition of a book first published in 1973. It is a history of the Catholic Church in England and Wales from the Reformation to the present day, and of the buildings and institutions in which it has been housed. It is a book about the Church and its houses, and not about the houses and their churches. The book is also a study of the changing face of Catholicism in England and Wales. The book is illustrated throughout with photographs of the buildings.<|end_of_text|>


In [27]:
model.save_pretrained("/content/drive/MyDrive/FIAP/TC4/tc4_model")
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/TC4/tc4_model")

('/content/drive/MyDrive/FIAP/TC4/tc4_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/TC4/tc4_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/TC4/tc4_model/tokenizer.json')

In [ ]:
# import json
# news_test = []

# news_test = json.loads(test_products.read())['test']

In [28]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/FIAP/TC4/tc4_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "GENERATE A DESCRIPTION FOR THIS PRODUCT",
        "Giant's Bread",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
GENERATE A DESCRIPTION FOR THIS PRODUCT

### Input:
Giant's Bread

### Response:
Giant's Bread is a 1953 British film directed by Albert R. Broccoli and starring Cedric Hardwicke and Marius Goring. It is based on the novel of the same name by Edith Pargeter, who later wrote the Sister Fidelma mysteries under the name of Ellis Peters.<|end_of_text|>
